## Load libraries

In [2]:
import json
import numpy as np
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
from companionCode.wrangling.parse import df_from_bib
from companionCode.visualization.categorical_plots import *
from matplotlib_venn import venn2


## Visualization options

In [3]:
# custom colors
base_c = [i / 255 for i in [153, 0, 0]]
comp_c = [i / 255 for i in [25, 176, 192]]
tri_1_c = [i / 255 for i in [25, 196, 49]]
tri_2_c = [i / 255 for i in [49, 25, 196]]
tet_1_c = [i / 255 for i in [87, 196, 25]]
tet_2_c = [i / 255 for i in [25, 172, 196]]
tet_3_c = [i / 255 for i in [135, 25, 196]]
my_gray = [i/ 255 for i in [104, 104, 104]]
# rendering option
matplotlib.rcParams["text.usetex"] = True
matplotlib.rcParams["font.family"] = "serif"
matplotlib.rcParams["font.size"] = "11"
# alpha params
alpha = [0.6]

## Scopus search outcome and coding outcome

### Read data

In [6]:
# Scopus's search outcome
df0 = df_from_bib("data/management_studies_on_ex_shocks_clean.bib")
# coded data
df1 = pd.read_csv("data/coded_studies.csv")
# merge data
df = pd.merge(df0, df1.loc[df1["sample"] == True], on="key", how="inner")
del df0, df1


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   key                          31 non-null     object
 1   journal                      31 non-null     object
 2   year                         31 non-null     int64 
 3   author                       31 non-null     object
 4   title                        31 non-null     object
 5   sample                       31 non-null     bool  
 6   event_description            31 non-null     object
 7   event_relevance              31 non-null     object
 8   event_class                  31 non-null     object
 9   role_description             31 non-null     object
 10  role_theory                  31 non-null     object
 11  role_empirical               31 non-null     object
 12  pre_treatmnet                31 non-null     object
 13  control_group                31 non-n

### Sample description

#### Distribution of studies across journals

In [9]:
chart = BarChart(name_="studies_across_journals")
chart.add_data(df_=df, var_="journal")
chart.plot(
    fig_size=(3.75, 2.5),
    use_tex=True,
    edge_color=my_gray,
    face_color=my_gray,
    tick_min=0,
    tick_step=2,
    axis_label="Counts of studies",
    horizontal_=True,
)



                  PGF is a non-GUI backend -- nothing to display here!
                  
                  To display the chart here, set 'use_tex' = False 
                  


### Type of naturally-occurring event

In [10]:
chart = BarChart(name_="classes_of_events")
chart.add_data(df_=df, var_="event_class")
chart.plot(
    fig_size=(3.75, 3.5),
    use_tex=True,
    face_color=my_gray,
    edge_color=my_gray,
    tick_min=0,
    tick_step=2,
    axis_label="Counts of studies",
    horizontal_=True,
)



                  PGF is a non-GUI backend -- nothing to display here!
                  
                  To display the chart here, set 'use_tex' = False 
                  


### Event-research question mapping

In [11]:
# get cross tab
cross_tab = pd.crosstab(df["role_empirical"], df["role_theory"])
# create figure
fig = plt.figure(figsize=(2, 2))
ax = fig.add_subplot(1, 1, 1)
venn2(
    subsets=(cross_tab[0][1], cross_tab[1][0], cross_tab[1][1]),
    set_labels=("Empirical role", "Substantive role"),
    set_colors=(my_gray, base_c),
    alpha=0.7,
)
plt.savefig("manuscript/exhibits/event_roles.pgf", bbox_inches="tight", pad_inches=0)


### Potential outcome framework

In [12]:
# data
# --+ event plays empirical role
gr = df.loc[(df["role_empirical"] == True) & (df["role_theory"] == False)].groupby(
    ["control_group"], as_index=False
)
c0 = pd.DataFrame(gr.size())
# --+ event plays substantive role
gr = df.loc[(df["role_empirical"] == False) & (df["role_theory"] == True)].groupby(
    ["control_group"], as_index=False
)
c1 = pd.DataFrame(gr.size())
# --+ event plays both roles
gr = df.loc[(df["role_empirical"] == True) & (df["role_theory"] == True)].groupby(
    ["control_group"], as_index=False
)
c2 = pd.DataFrame(gr.size())


In [17]:
# create stacked bar chart
fig = plt.figure(figsize=(2.5, 0.9))
ax = fig.add_subplot(1, 1, 1)
ax.barh(
    c0.index,
    c0["size"],
    height=0.25,
    label="Empirical",
    color=my_gray
)
ax.barh(
    c2.index,
    c2["size"],
    left=c0["size"],
    height=0.25,
    label="Empirical \& substantive",
    color=my_gray,
    alpha=0.7
)
ax.barh(
    c1.index,
    c1["size"],
    height=0.25,
    left=c0["size"] + c2["size"],
    label="Substantive",
    color=my_gray,
    alpha=0.4
)
# labels and ticks
ax.set_yticks([0, 1])
ax.set_yticklabels(["Design lacking a control group", "Design with a control group"])
ax.set_xlabel("Counts of studies")
# hide the right and top spines
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.yaxis.set_ticks_position("left")
ax.xaxis.set_ticks_position("bottom")
# legend
ax.legend(loc=(1, 0), frameon=False)
# tick labels' jsutification
plt.setp(
    ax.get_yticklabels(),
    rotation=0,
    ha="right",
    va="center",
    rotation_mode="anchor",
)
# plot
plt.savefig("manuscript/exhibits/potential_outcome.pgf", bbox_inches="tight", pad_inches=0)
